This notebook provides a graphical user interface (GUI) for reviewing and labeling various object classes in annotated satellite imagery. Users can inspect cropped image regions, apply quality control (QC) labels, and save changes. It's designed to support multi-class annotation such as:
PV_normal

PV_heater

PV_pool

PV_heater_mat

Uncertain

Deleted

Resizing needed



Import Required Libraries

In [1]:
import geopandas as gpd
import rasterio
from PIL import Image, ImageTk, ImageDraw
import tkinter as tk
from tkinter import ttk, messagebox
import numpy as np
import os


Set File Paths and Load Data

In [ ]:
# Modify these paths based on your system setup
GPKG_PATH = '/path/to/your/annotations.gpkg'           # Input annotation file
IMAGE_FOLDER = '/path/to/your/images'                  # Folder with .tif images
OUTPUT_PATH = '/path/to/save/updated_annotations.gpkg' # Output GPKG after QC

# Load the annotation dataset
gdf = gpd.read_file(GPKG_PATH)

Ensure QC Columns Exist

In [ ]:
# List of QC columns corresponding to different object classes and actions
qc_cols = ["PV_normal_qc", "PV_heater_qc", "PV_pool_qc", 
           "PV_heater_mat", "uncertflag_qc", "delete_qc", "resizing_qc"]

# Add any missing columns with default value 0
for col in qc_cols:
    if col not in gdf.columns:
        gdf[col] = 0


Define the QCChecker Class


This class handles GUI layout, image loading, user interaction, and data saving.




In [ ]:
class QCChecker:
    def __init__(self, master):
        self.master = master
        self.index = self.find_next_unchecked_index()
        self.resizing_selected = False
        self.selected_qcs = set()
        self.zoom_scale = 1.0

        # Layout: original + annotated image side-by-side
        self.label_frame = tk.Frame(master)
        self.label_frame.pack()
        self.label_original = tk.Label(self.label_frame)
        self.label_original.pack(side=tk.LEFT, padx=5)
        self.label_annotated = tk.Label(self.label_frame)
        self.label_annotated.pack(side=tk.LEFT, padx=5)

        # Info box
        self.info = tk.Label(master, text="", font=("Arial", 12), justify="left")
        self.info.pack()

        # QC label buttons
        button_frame = ttk.Frame(master)
        button_frame.pack()
        for i, col in enumerate(qc_cols):
            ttk.Button(button_frame, text=col, command=lambda c=col: self.mark(c)).grid(row=0, column=i, padx=4)

        # Zoom buttons
        zoom_frame = ttk.Frame(master)
        zoom_frame.pack(pady=5)
        ttk.Button(zoom_frame, text="Zoom +", command=self.zoom_in).pack(side=tk.LEFT, padx=5)
        ttk.Button(zoom_frame, text="Zoom -", command=self.zoom_out).pack(side=tk.LEFT, padx=5)

        self.load_image()


Navigate Annotations

In [ ]:
    def find_next_unchecked_index(self):
        # Return first un-reviewed annotation
        for i, row in gdf.iterrows():
            if not any(row[qc] == 1 for qc in qc_cols):
                return i
        print("All annotations have been reviewed.")
        return 0


Mark and Save QC Labels

In [ ]:
    def mark(self, col):
        # If the label is a special action (uncertain or resizing), mark it and wait for a class label
        if col in {"resizing_qc", "uncertflag_qc"}:
            self.selected_qcs = {col}
            self.resizing_selected = True
            if col == "uncertflag_qc":
                self.save_current_images_as_png()
            print(f"{col} selected. Now select the appropriate object label.")
            return

        elif self.resizing_selected:
            # If second part of labeling is selected (after uncertain/resizing)
            self.selected_qcs.add(col)
            if "uncertflag_qc" in self.selected_qcs:
                self.save_current_images_as_png()
            self.save_and_advance()
            self.advance_or_quit()
            return

        else:
            # Single-label QC
            self.selected_qcs = {col}
            self.save_and_advance()
            self.advance_or_quit()


Save QC Labels to File

In [ ]:
    def save_and_advance(self):
        for qc in qc_cols:
            gdf.at[self.index, qc] = 1 if qc in self.selected_qcs else 0
        try:
            gdf.to_file(OUTPUT_PATH, driver="GPKG")
            print(f"Saved at ID {gdf.iloc[self.index].get('id', self.index)}")
        except Exception as e:
            print(f"Save failed: {e}")
        self.selected_qcs = set()
        self.resizing_selected = False


Move to Next or Exit

In [ ]:
    def advance_or_quit(self):
        if self.index < len(gdf) - 1:
            self.index += 1
            self.load_image()
        else:
            messagebox.showinfo("Done", "QC annotations are complete!")
            self.master.quit()


Save Image Pair to PNG (Optional)

In [ ]:
    def save_current_images_as_png(self, save_dir="saved_screens"):
        os.makedirs(save_dir, exist_ok=True)
        row = gdf.iloc[self.index]
        image_id = row.get("id", self.index)

        original_path = os.path.join(save_dir, f"{image_id}_original.png")
        annotated_path = os.path.join(save_dir, f"{image_id}_annotated.png")

        try:
            self.img_original_pil.save(original_path, "PNG")
            self.img_annotated_pil.save(annotated_path, "PNG")
            print(f"Saved images for ID {image_id}")
        except Exception as e:
            print(f"Failed to save images for ID {image_id}: {e}")


Zoom function

In [ ]:
    def zoom_in(self):
        self.zoom_scale /= 1.2
        self.load_image()

    def zoom_out(self):
        self.zoom_scale *= 1.2
        self.load_image()


Load and Display Image Region

In [ ]:
    def load_image(self):
        row = gdf.iloc[self.index]
        image_name = row.get("image_name")
        image_path = os.path.join(IMAGE_FOLDER, image_name + ".tif")

        try:
            with rasterio.open(image_path) as src:
                geom = row.geometry
                transform = src.transform
                centroid = geom.centroid
                cx, cy = ~transform * (centroid.x, centroid.y)

                # Define crop box around centroid
                base_crop = 300
                half_w = int(base_crop * self.zoom_scale)
                half_h = int(base_crop * self.zoom_scale)

                box_crop = (
                    max(0, int(cx - half_w)),
                    max(0, int(cy - half_h)),
                    min(src.width, int(cx + half_w)),
                    min(src.height, int(cy + half_h))
                )

                # Extract image window
                window = rasterio.windows.Window(
                    col_off=box_crop[0],
                    row_off=box_crop[1],
                    width=box_crop[2] - box_crop[0],
                    height=box_crop[3] - box_crop[1]
                )

                data = src.read([1, 2, 3], window=window)
                win_transform = src.window_transform(window)

                rgb = np.transpose(data, (1, 2, 0))
                rgb = np.nan_to_num(rgb)
                if rgb.dtype != np.uint8:
                    rgb = ((rgb - rgb.min()) / (rgb.ptp() + 1e-6) * 255).astype(np.uint8)

                # Prepare original and annotated images
                img_original = Image.fromarray(rgb)
                img_annotated = img_original.copy()

                draw = ImageDraw.Draw(img_annotated)
                if hasattr(geom, "exterior"):
                    coords = list(geom.exterior.coords)
                    pixels = [~win_transform * (x, y) for x, y in coords]
                    pixels = [(int(x), int(y)) for x, y in pixels]
                    if len(pixels) > 2:
                        draw.polygon(pixels, outline="red", width=3)

                # Resize images if needed
                for img in [img_original, img_annotated]:
                    if img.width > 800 or img.height > 800:
                        img.thumbnail((800, 800), Image.LANCZOS)

                # Convert to Tkinter images
                self.img_original_pil = img_original
                self.img_annotated_pil = img_annotated
                self.tk_img_original = ImageTk.PhotoImage(img_original)
                self.tk_img_annotated = ImageTk.PhotoImage(img_annotated)

                # Display images in GUI
                self.label_original.configure(image=self.tk_img_original)
                self.label_annotated.configure(image=self.tk_img_annotated)

                self.label_original.image = self.tk_img_original
                self.label_annotated.image = self.tk_img_annotated

                # Show metadata
                self.info.config(
                    text=(
                        f"ID: {row.get('id', 'NA')} | image: {image_name} | annotator: {row.get('annotator', 'NA')}\n"
                        f"PV_normal: {row.get('PV_normal')}, "
                        f"PV_heater: {row.get('PV_heater')}, "
                        f"PV_pool: {row.get('PV_pool')}, "
                        f"uncertflag: {row.get('uncertflag')}"
                    )
                )
        except Exception as e:
            print(f"Error loading image {image_path}: {e}")


Launch the GUI

In [ ]:
# Run the GUI
root = tk.Tk()
root.title("Multi-Class Annotation QC Checker")
app = QCChecker(root)
root.mainloop()
